In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split 

## Преобразование колонок различных типов

In [78]:
import pandas as pd
X = pd.DataFrame(
    {'city': ['London', 'London', 'Paris', 'Sallisaw'],
     'title': ["His Last Bow", "How Watson Learned the Trick",
               "A Moveable Feast", "The Grapes of Wrath"],
     'expert_rating': [5, 3, 4, 5],
     'user_rating': [4, 5, 4, 3]})
X

,city,title,expert_rating,user_rating
0,London,His Last Bow,5,4
1,London,How Watson Learned the Trick,3,5
2,Paris,A Moveable Feast,4,4
3,Sallisaw,The Grapes of Wrath,5,3


In [80]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector


ct = ColumnTransformer([
      ('scale', StandardScaler(),
      make_column_selector(dtype_include=np.number)),
      ('onehot',
      OneHotEncoder(),
      make_column_selector(pattern='city', dtype_include=object))
      ])

ct.fit_transform(X)

array([[ 0.90453403,  0.        ,  1.        ,  0.        ,  0.        ],
       [-1.50755672,  1.41421356,  1.        ,  0.        ,  0.        ],
       [-0.30151134,  0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.90453403, -1.41421356,  0.        ,  0.        ,  1.        ]])

In [82]:
ct.get_feature_names_out()

array(['scale__expert_rating', 'scale__user_rating',
       'onehot__city_London', 'onehot__city_Paris',
       'onehot__city_Sallisaw'], dtype=object)

In [84]:
pd.DataFrame(data=ct.fit_transform(X), columns = ct.get_feature_names_out())

,scale__expert_rating,scale__user_rating,onehot__city_London,onehot__city_Paris,onehot__city_Sallisaw
0,0.904534,0.000000,1.0,0.0,0.0
1,-1.507557,1.414214,1.0,0.0,0.0
2,-0.301511,0.000000,0.0,1.0,0.0
3,0.904534,-1.414214,0.0,0.0,1.0


In [86]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector

display(X)

ct_r = ColumnTransformer([
      ('scale', StandardScaler(),
      make_column_selector(pattern='expert_rating')),
      ('onehot',
      OneHotEncoder(),
      make_column_selector(pattern='city', dtype_include=object)),
      ('drop_colunm',
       'drop',
      make_column_selector(pattern='title', dtype_include=object))
      ],
      remainder='passthrough')

X_cl = ct_r.fit_transform(X)

pd.DataFrame(X_cl, columns = ct_r.get_feature_names_out()) 

,city,title,expert_rating,user_rating
0,London,His Last Bow,5,4
1,London,How Watson Learned the Trick,3,5
2,Paris,A Moveable Feast,4,4
3,Sallisaw,The Grapes of Wrath,5,3


,scale__expert_rating,onehot__city_London,onehot__city_Paris,onehot__city_Sallisaw,remainder__user_rating
0,0.904534,1.0,0.0,0.0,4.0
1,-1.507557,1.0,0.0,0.0,5.0
2,-0.301511,0.0,1.0,0.0,4.0
3,0.904534,0.0,0.0,1.0,3.0


In [88]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler

display(X)

column_trans = make_column_transformer(
    (OneHotEncoder(), ['city']),
    ('drop', 'title'),
    remainder=MinMaxScaler())

display(column_trans)

z =column_trans.fit_transform(X)

pd.DataFrame(z, columns = column_trans.get_feature_names_out()) 

,city,title,expert_rating,user_rating
0,London,His Last Bow,5,4
1,London,How Watson Learned the Trick,3,5
2,Paris,A Moveable Feast,4,4
3,Sallisaw,The Grapes of Wrath,5,3


ColumnTransformer(remainder=MinMaxScaler(),
                  transformers=[('onehotencoder', OneHotEncoder(), ['city']),
                                ('drop', 'drop', 'title')])

,onehotencoder__city_London,onehotencoder__city_Paris,onehotencoder__city_Sallisaw,remainder__expert_rating,remainder__user_rating
0,1.0,0.0,0.0,1.0,0.5
1,1.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.5,0.5
3,0.0,0.0,1.0,1.0,0.0


In [90]:
X_col_tr = column_trans.fit_transform(X)
pd.DataFrame(X_col_tr , columns = column_trans.get_feature_names_out()) 

,onehotencoder__city_London,onehotencoder__city_Paris,onehotencoder__city_Sallisaw,remainder__expert_rating,remainder__user_rating
0,1.0,0.0,0.0,1.0,0.5
1,1.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.5,0.5
3,0.0,0.0,1.0,1.0,0.0


## Сложные преобразования и заполнения пропущенных данных

In [93]:
df = pd.read_csv('Diamants/dim_train.csv')
df.tail()

,carat,cut,color,clarity,depth,table,price,x,y,z
37753,0.33,Premium,G,SI1,59.8,58.0,631,4.54,4.52,2.71
37754,2.01,Fair,F,SI1,58.6,95.0,13387,8.32,8.31,4.87
37755,0.26,Ideal,E,VVS2,61.5,56.0,769,4.11,4.08,2.52
37756,0.40,Very Good,G,SI1,62.8,60.0,702,4.66,4.70,2.94
37757,0.30,Ideal,D,SI1,62.4,54.0,508,4.32,4.34,2.70


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37758 entries, 0 to 37757
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    37758 non-null  float64
 1   cut      37758 non-null  object 
 2   color    37758 non-null  object 
 3   clarity  37758 non-null  object 
 4   depth    37758 non-null  float64
 5   table    37758 non-null  float64
 6   price    37758 non-null  int64  
 7   x        37758 non-null  float64
 8   y        37758 non-null  float64
 9   z        37758 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 2.9+ MB


In [97]:
df.describe()

,carat,depth,table,price,x,y,z
count,37758.000000,37758.000000,37758.000000,37758.000000,37758.000000,37758.000000,37758.000000
mean,0.798777,61.755085,57.460784,3932.113300,5.732871,5.736693,3.539717
std,0.474574,1.440063,2.230229,3980.261121,1.122451,1.154608,0.695749
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,953.000000,4.720000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2404.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5345.000000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,8.060000


In [99]:
# отбор всех сторок с нулевым. значением в любом месте колонки
df[df.eq(0).any(axis=1)]

,carat,cut,color,clarity,depth,table,price,x,y,z
2949,1.07,Ideal,F,SI2,61.6,56.0,4954,0.00,6.62,0.0
6374,1.01,Premium,F,SI2,59.2,58.0,3837,6.50,6.47,0.0
8075,1.00,Very Good,H,VS2,63.3,53.0,5139,0.00,0.00,0.0
8909,2.80,Good,G,SI2,63.8,58.0,18788,8.90,8.85,0.0
9673,2.18,Premium,H,SI2,59.4,61.0,12631,8.49,8.45,0.0
12121,0.71,Good,F,SI2,64.1,60.0,2130,0.00,0.00,0.0
18790,1.10,Premium,G,SI2,63.0,59.0,3696,6.50,6.47,0.0
20612,0.71,Good,F,SI2,64.1,60.0,2130,0.00,0.00,0.0
21727,2.25,Premium,I,SI1,61.3,58.0,15397,8.52,8.42,0.0
25252,1.15,Ideal,G,VS2,59.2,56.0,5564,6.88,6.83,0.0


In [101]:
df.describe()

,carat,depth,table,price,x,y,z
count,37758.000000,37758.000000,37758.000000,37758.000000,37758.000000,37758.000000,37758.000000
mean,0.798777,61.755085,57.460784,3932.113300,5.732871,5.736693,3.539717
std,0.474574,1.440063,2.230229,3980.261121,1.122451,1.154608,0.695749
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,953.000000,4.720000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2404.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5345.000000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,8.060000


In [103]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.52,Ideal,J,SI1,61.9,57.0,7491,7.37,7.33,4.55
1,0.31,Very Good,F,SI2,63.0,58.0,408,4.34,4.36,2.74
2,1.00,Premium,F,SI1,60.7,60.0,5292,6.40,6.36,3.87
3,1.01,Very Good,E,VVS2,63.3,58.0,8912,6.39,6.31,4.02
4,0.40,Premium,I,SI2,62.9,59.0,585,4.68,4.63,2.93


Выделим отдельно категориальные фичи и те, к которым мы хотим сделать полиномиальный фичинг. Также отдельно остивим фичи, где будем применять StandartScaler. Также выделим целевую переменную

In [106]:
cat_features = ['cut','color']
axis_features = ['x','y','z']
num_features = ['carat','depth','table']
y = np.array(df.price)
X = df.drop(columns=['price'])

Разделим на train/test

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Расширенные преобразования в энкодоре

In [112]:
#### Нам нужна библиотека для строгой иерархической перекодировки https://contrib.scikit-learn.org/category_encoders/
#pip install category_encoders

In [114]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
import category_encoders as ce

Следующие три блока делают одно и тоже, просто признаки ставновятся в разный порядок, если их позапускать по отдельности, можно получить совершено разные результаты

In [117]:
import category_encoders as ce

axis_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', StandardScaler())])

clarity_map = [{
    'col':'clarity',##### Обратить внимание
    'mapping':{'FL':10, 'IF':9, 'VVS1':8, 'VVS2':7, 'VS1':6, 'VS2':5, 
               'SI1':4, 'SI2':3, 'I1':2, 'I2':1, 'I3':0}
    }]

clarity_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=clarity_map)),
    ('scaler', MinMaxScaler())
    ])
    

CT = ColumnTransformer([
        ("pol_std", axis_transformer, axis_features),
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(), cat_features),
        ("ordinal_map", clarity_transformer, ['clarity'])##### Обратить внимание
        ])

display(CT)

res_ct = CT.fit_transform(X_train)
pd.DataFrame(res_ct).head()


ColumnTransformer(transformers=[('pol_std',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['x', 'y', 'z']),
                                ('num', StandardScaler(),
                                 ['carat', 'depth', 'table']),
                                ('cat', OneHotEncoder(), ['cut', 'color']),
                                ('ordinal_map',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'clarity',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': FL      10
IF       9
VVS1     8
VVS2     7
VS1      6
VS2      5
SI1      4
SI2      3
I1       2
I2       1
I3       0
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['clarity'])])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.054693,0.987674,1.019010,1.023372,0.989584,1.007947,0.876382,0.970174,0.984246,0.885971,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.571429
1,-0.398512,-0.426609,-0.549765,-0.461659,-0.475040,-0.529659,-0.450719,-0.541102,-0.594914,-0.570317,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428571
2,-1.156318,-1.182671,-1.183032,-1.078771,-1.091890,-1.091665,-1.017230,-1.100850,-1.100752,-0.992430,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714
3,0.626755,0.649669,0.601629,0.544874,0.560505,0.534610,0.527522,0.547761,0.518742,0.442753,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.571429
4,-1.236556,-1.271620,-1.226209,-1.137800,-1.154129,-1.137126,-1.077330,-1.149404,-1.132485,-1.034641,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857


In [119]:
X_train.describe()

,carat,depth,table,x,y,z
count,28318.000000,28318.000000,28318.000000,28318.000000,28318.000000,28318.000000
mean,0.800220,61.751038,57.467547,5.736994,5.739613,3.541982
std,0.473816,1.445905,2.230641,1.121679,1.124265,0.694822
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,4.720000,4.730000,2.920000
50%,0.700000,61.800000,57.000000,5.700000,5.710000,3.530000
75%,1.050000,62.500000,59.000000,6.550000,6.540000,4.040000
max,5.010000,79.000000,95.000000,10.740000,31.800000,6.980000


## Стратегии заполнения пропусков в данных

Особено полезно, кода на вход в моделе могут быть пропущенные или ошибочные данные, как мы видим в этом наборе.

In [122]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

In [124]:
ind_nan = X_train[X_train.eq(0).any(axis=1)].index
X_train[X_train.eq(0).any(axis=1)]

,carat,cut,color,clarity,depth,table,x,y,z
25252,1.15,Ideal,G,VS2,59.2,56.0,6.88,6.83,0.0
27985,1.12,Premium,G,I1,60.4,59.0,6.71,6.67,0.0
12121,0.71,Good,F,SI2,64.1,60.0,0.00,0.00,0.0
18790,1.10,Premium,G,SI2,63.0,59.0,6.50,6.47,0.0
28349,1.56,Ideal,G,VS2,62.2,54.0,0.00,0.00,0.0
34835,1.14,Fair,G,VS1,57.5,67.0,0.00,0.00,0.0
20612,0.71,Good,F,SI2,64.1,60.0,0.00,0.00,0.0
8075,1.00,Very Good,H,VS2,63.3,53.0,0.00,0.00,0.0
9673,2.18,Premium,H,SI2,59.4,61.0,8.49,8.45,0.0
21727,2.25,Premium,I,SI1,61.3,58.0,8.52,8.42,0.0


In [126]:
ind_nan = X_train[X_train.eq(0).any(axis=1)].index
X_train[X_train.eq(0).any(axis=1)]

,carat,cut,color,clarity,depth,table,x,y,z
25252,1.15,Ideal,G,VS2,59.2,56.0,6.88,6.83,0.0
27985,1.12,Premium,G,I1,60.4,59.0,6.71,6.67,0.0
12121,0.71,Good,F,SI2,64.1,60.0,0.00,0.00,0.0
18790,1.10,Premium,G,SI2,63.0,59.0,6.50,6.47,0.0
28349,1.56,Ideal,G,VS2,62.2,54.0,0.00,0.00,0.0
34835,1.14,Fair,G,VS1,57.5,67.0,0.00,0.00,0.0
20612,0.71,Good,F,SI2,64.1,60.0,0.00,0.00,0.0
8075,1.00,Very Good,H,VS2,63.3,53.0,0.00,0.00,0.0
9673,2.18,Premium,H,SI2,59.4,61.0,8.49,8.45,0.0
21727,2.25,Premium,I,SI1,61.3,58.0,8.52,8.42,0.0


### Заполнение нужным значением среднем или наиболее частым `SimpleImputer`

Используем, чтобы не исказить статистику по данным, но может дать в итоге ошибки в модели. Заполняется единым значением по всем "дыркам".

In [129]:
column_trans = make_column_transformer(
    (SimpleImputer(missing_values=0, strategy='mean'), ['x','y','z'])
     )

X_ct = column_trans.fit_transform(X_train)

df_ct = pd.DataFrame(X_ct, columns = column_trans.get_feature_names_out(), index = X_train.index)
display(df_ct.head())
df_ct.describe()

,simpleimputer__x,simpleimputer__y,simpleimputer__z
25885,6.92,6.85,4.25
1804,5.29,5.26,3.16
1844,4.44,4.41,2.72
27396,6.44,6.47,3.96
24691,4.35,4.31,2.69


,simpleimputer__x,simpleimputer__y,simpleimputer__z
count,28318.000000,28318.000000,28318.000000
mean,5.738008,5.740626,3.543359
std,1.119085,1.121675,0.691305
min,3.740000,3.710000,1.070000
25%,4.720000,4.730000,2.920000
50%,5.700000,5.710000,3.530000
75%,6.550000,6.540000,4.040000
max,10.740000,31.800000,6.980000


In [131]:
pd.concat([X_train[X_train.eq(0).any(axis=1)][['x','y','z']], df_ct.loc[ind_nan] ], axis=1)

,x,y,z,simpleimputer__x,simpleimputer__y,simpleimputer__z
25252,6.88,6.83,0.0,6.880000,6.830000,3.543359
27985,6.71,6.67,0.0,6.710000,6.670000,3.543359
12121,0.00,0.00,0.0,5.738008,5.740626,3.543359
18790,6.50,6.47,0.0,6.500000,6.470000,3.543359
28349,0.00,0.00,0.0,5.738008,5.740626,3.543359
34835,0.00,0.00,0.0,5.738008,5.740626,3.543359
20612,0.00,0.00,0.0,5.738008,5.740626,3.543359
8075,0.00,0.00,0.0,5.738008,5.740626,3.543359
9673,8.49,8.45,0.0,8.490000,8.450000,3.543359
21727,8.52,8.42,0.0,8.520000,8.420000,3.543359


### Заполнение итеративным методом

Заполняем пропуски моделируя функцию с отсутствующими значениями как функцию других функций и использует эту оценку для заполнения. Алгоритм делает это в итерированном круговом режиме: на каждом шаге столбец признаков обозначается как вывод y, а другие столбцы признаков рассматриваются как входы X. Регрессор помещается на (X, y) для известных y. Затем регрессор используется для прогнозирования недостающих значений y. Это делается для каждой функции итеративно, а затем повторяется для раундов max_iter.

Могут быть разные способы заполнения. По умолчанию байесовская регрессия. [Возможен любой алгоритм.](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer)

In [134]:
column_trans = make_column_transformer(
    (IterativeImputer(missing_values=0, max_iter=20, random_state=0), ['carat','x','y','z']) 
     )

X_ct = column_trans.fit_transform(X_train)

df_it = pd.DataFrame(X_ct, columns = column_trans.get_feature_names_out(), index = X_train.index)
display(df_it.head())
df_it.describe()

,iterativeimputer__carat,iterativeimputer__x,iterativeimputer__y,iterativeimputer__z
25885,1.22,6.92,6.85,4.25
1804,0.53,5.29,5.26,3.16
1844,0.33,4.44,4.41,2.72
27396,1.01,6.44,6.47,3.96
24691,0.31,4.35,4.31,2.69


,iterativeimputer__carat,iterativeimputer__x,iterativeimputer__y,iterativeimputer__z
count,28318.000000,28318.000000,28318.000000,28318.000000
mean,0.800220,5.738097,5.740715,3.543613
std,0.473816,1.119145,1.121734,0.691530
min,0.200000,3.740000,3.710000,1.070000
25%,0.400000,4.720000,4.730000,2.920000
50%,0.700000,5.700000,5.710000,3.530000
75%,1.050000,6.550000,6.540000,4.040000
max,5.010000,10.740000,31.800000,6.980000


In [136]:
pd.concat([X_train[X_train.eq(0).any(axis=1)][['x','y','z']], df_ct.loc[ind_nan], df_it.loc[ind_nan] ], axis=1)

,x,y,z,simpleimputer__x,simpleimputer__y,simpleimputer__z,iterativeimputer__carat,iterativeimputer__x,iterativeimputer__y,iterativeimputer__z
25252,6.88,6.83,0.0,6.880000,6.830000,3.543359,1.15,6.880000,6.830000,4.208342
27985,6.71,6.67,0.0,6.710000,6.670000,3.543359,1.12,6.710000,6.670000,4.115066
12121,0.00,0.00,0.0,5.738008,5.740626,3.543359,0.71,5.533920,5.538499,3.417214
18790,6.50,6.47,0.0,6.500000,6.470000,3.543359,1.10,6.500000,6.470000,4.003881
28349,0.00,0.00,0.0,5.738008,5.740626,3.543359,1.56,7.457551,7.443650,4.608074
34835,0.00,0.00,0.0,5.738008,5.740626,3.543359,1.14,6.507051,6.502281,4.019649
20612,0.00,0.00,0.0,5.738008,5.740626,3.543359,0.71,5.533920,5.538499,3.417214
8075,0.00,0.00,0.0,5.738008,5.740626,3.543359,1.00,6.190217,6.188492,3.823507
9673,8.49,8.45,0.0,8.490000,8.450000,3.543359,2.18,8.490000,8.450000,5.286578
21727,8.52,8.42,0.0,8.520000,8.420000,3.543359,2.25,8.520000,8.420000,5.314029


### Заполнение методом ближайших соседей "дырок" в данных

Специализированное решение ближайшими соседями.

In [139]:
column_trans = make_column_transformer(
    (KNNImputer(missing_values=0, n_neighbors=3, weights="uniform"), ['carat','x','y','z']) 
     )

X_ct = column_trans.fit_transform(X_train)

df_knn = pd.DataFrame(X_ct, columns = column_trans.get_feature_names_out(), index = X_train.index)
display(df_it.head())
df_knn.describe()

,iterativeimputer__carat,iterativeimputer__x,iterativeimputer__y,iterativeimputer__z
25885,1.22,6.92,6.85,4.25
1804,0.53,5.29,5.26,3.16
1844,0.33,4.44,4.41,2.72
27396,1.01,6.44,6.47,3.96
24691,0.31,4.35,4.31,2.69


,knnimputer__carat,knnimputer__x,knnimputer__y,knnimputer__z
count,28318.000000,28318.000000,28318.000000,28318.000000
mean,0.800220,5.738122,5.740738,3.543612
std,0.473816,1.119153,1.121741,0.691493
min,0.200000,3.740000,3.710000,1.070000
25%,0.400000,4.720000,4.730000,2.920000
50%,0.700000,5.700000,5.710000,3.530000
75%,1.050000,6.550000,6.540000,4.040000
max,5.010000,10.740000,31.800000,6.980000


In [141]:
pd.concat([X_train[X_train.eq(0).any(axis=1)][['x','y','z']], df_it.loc[ind_nan], df_knn.loc[ind_nan] ], axis=1)

,x,y,z,iterativeimputer__carat,iterativeimputer__x,iterativeimputer__y,iterativeimputer__z,knnimputer__carat,knnimputer__x,knnimputer__y,knnimputer__z
25252,6.88,6.83,0.0,1.15,6.880000,6.830000,4.208342,1.15,6.880000,6.830000,4.113333
27985,6.71,6.67,0.0,1.12,6.710000,6.670000,4.115066,1.12,6.710000,6.670000,4.106667
12121,0.00,0.00,0.0,0.71,5.533920,5.538499,3.417214,0.71,5.716667,5.726667,3.553333
18790,6.50,6.47,0.0,1.10,6.500000,6.470000,4.003881,1.10,6.500000,6.470000,4.080000
28349,0.00,0.00,0.0,1.56,7.457551,7.443650,4.608074,1.56,7.456667,7.463333,4.556667
34835,0.00,0.00,0.0,1.14,6.507051,6.502281,4.019649,1.14,6.783333,6.753333,4.090000
20612,0.00,0.00,0.0,0.71,5.533920,5.538499,3.417214,0.71,5.716667,5.726667,3.553333
8075,0.00,0.00,0.0,1.00,6.190217,6.188492,3.823507,1.00,6.260000,6.206667,3.986667
9673,8.49,8.45,0.0,2.18,8.490000,8.450000,5.286578,2.18,8.490000,8.450000,5.016667
21727,8.52,8.42,0.0,2.25,8.520000,8.420000,5.314029,2.25,8.520000,8.420000,5.160000
